In [1]:
import pandas as pd

#Read the training file
train=pd.read_csv("C:/Users/sanketn/Documents/IU/Kaggle/Project3/train.csv")
train.head(3)

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul


In [2]:
#Read the test file
test=pd.read_csv("C:/Users/sanketn/Documents/IU/Kaggle/Project3/test.csv")
test.head(3)

,id,bone_length,rotting_flesh,hair_length,has_soul,color
0,3,0.471774,0.387937,0.706087,0.698537,black
1,6,0.427332,0.645024,0.565558,0.451462,white
2,9,0.549602,0.491931,0.660387,0.449809,black


In [3]:
#File counts
print("Training count is %i." % train.shape[0])
print("Testing count is %i." % test.shape[0])

Training count is 371.
Testing count is 529.


In [4]:
#let's process missing values
def countMissing(data):
    missing = data.columns[data.isnull().any()].tolist()
    return missing
misTrain = countMissing(train)
misTest = countMissing(test)
misTotal = list(set().union(misTrain, misTest))

In [5]:
#No missing values
print(misTrain, len(misTrain))

[] 0


In [6]:
#Change alphanumeric color values to numeric values
mapping = {'black': 0, 'white': 1, 'green': 2, 'blue': 3, 'clear': 4, 'blood': 5}
train['color'] = train['color'].map(mapping)
test['color'] = test['color'].map(mapping)

In [7]:
#No missing values
print(misTrain, len(misTrain))

[] 0


In [8]:
#Remove ID from train and test sets.
X_train = train.drop(['id','type'], axis = 1)
Y_train = train['type']
X_test = test.drop(['id'], axis = 1)

In [10]:
#Use Gradient Boosting Classifier
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate=0.05, n_estimators=100,max_depth=2,random_state=0)
scores = cross_validation.cross_val_score(gbc, X_train, Y_train, scoring="accuracy", cv = 5, n_jobs = -1).mean()
scores

C:\Users\sanketn\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.7330252992718747

In [11]:
#Optimize parameters for LogisticRegression
from sklearn import linear_model
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
alphas = [0.01, 0.03, 0.1, 0.3, 1.0, 3.0, 5.0, 10]
regs = ["l1", "l2"]
scores = []
param = []
for alpha in alphas:
    for reg in regs:
        lm = linear_model.LogisticRegression(penalty = reg, C = alpha)
        scores.append(cross_validation.cross_val_score(lm, X_train, Y_train, scoring="accuracy", cv = 10).mean())
        param.append([alpha, reg])
scores = pd.DataFrame({'parameter': param, 'score': scores})
print(scores.sort_values(by = 'score', ascending = False))

     parameter     score
14    [10, l1]  0.751418
12   [5.0, l1]  0.751272
10   [3.0, l1]  0.748641
15    [10, l2]  0.737972
13   [5.0, l2]  0.737755
8    [1.0, l1]  0.732492
11   [3.0, l2]  0.724383
6    [0.3, l1]  0.713493
9    [1.0, l2]  0.705528
7    [0.3, l2]  0.692287
5    [0.1, l2]  0.622762
4    [0.1, l1]  0.545510
3   [0.03, l2]  0.480130
1   [0.01, l2]  0.380509
0   [0.01, l1]  0.334401
2   [0.03, l1]  0.331623


In [12]:
#Predict using optimized parameters
res = linear_model.LogisticRegression(penalty = "l1", C = 10.0).fit(X_train, Y_train).predict(X_test)

In [13]:
#Combine the predicted values to test set 
df = pd.DataFrame(data=res, columns=['type'])
df['id'] = test['id']
df

,type,id
0,Ghoul,3
1,Ghoul,6
2,Ghoul,9
3,Ghost,10
4,Ghost,13
5,Ghost,14
6,Ghoul,15
7,Ghoul,16
8,Goblin,17
9,Ghoul,18
